# Walk-through FitLins Analysis

This document demonstrates the command line to run a FitLins analysis.

We use the [OpenNeuro](https://openneuro.org) dataset [ds000003](https://openneuro.org/datasets/ds000003/),
with [fMRIPrep](https://fmriprep.rtfd.io)-preprocessed results at
[///labs/poldrack/ds003_fmriprep](http://datasets.datalad.org/?dir=/labs/poldrack/ds003_fmriprep).

## Preamble

We will use [Datalad](https://www.datalad.org) to manage our data, which in turn depends on `git-annex`. To install `git-annex`, uncomment and run one of the following:

In [ ]:
# Debian
# apt-get install git-annex

In [ ]:
# Linux with Conda
# conda install -y git-annex

In [ ]:
# OSX
# open https://git-annex.branchable.com/install/OSX/

Datalad and FitLins are Python tools.
To ensure that everything installs correctly, we'll upgrade the [pip](https://pip.pypa.io/en/stable/) package manager
and the [setuptools](https://setuptools.readthedocs.io/en/latest/) utilities.

We will also install unreleased versions of Datalad and [Nistats](https://nistats.github.io/) to make things run a little more smoothly.

In [ ]:
pip install -q --upgrade pip setuptools
pip install -q --upgrade "datalad @ git+https://github.com/datalad/datalad@5f5342ce2e63b91d54f75edb87d97129a916d9ef" \
                         "nistats @ git+https://github.com/nistats/nistats.git@b69e127020df05aa8ba540f9a8d799e1dad3d602"

## Install data and FitLins

First, install the [dataset](http://datasets.datalad.org/?dir=/labs/poldrack/ds003_fmriprep) we'll be working with.

In [ ]:
# Set environment variable to make outputs more friendly
export DATALAD_UI_PROGRESSBAR=log
export NO_COLOR="1"

datalad install -r ///labs/poldrack/ds003_fmriprep
datalad update ds003_fmriprep

Fetch the preprocessed BOLD series and calculated confounds for the all subjects.

In [ ]:
datalad get ds003_fmriprep/sub-*/func/*_space-MNI152NLin2009cAsym_desc-*.nii.gz \
            ds003_fmriprep/sub-*/func/*_desc-confounds_*.tsv

Finally, install FitLins via pip. See the [Installation](https://fitlins.readthedocs.io/en/latest/installation.html)
section of the documentation for instructions on using Docker or Singularity. For the purposes of this tutorial,
however, we'll assume you can install to your current Python environment without issue.

In [ ]:
pip install -q --upgrade fitlins==0.4.0

## Dataset structure

This dataset is organized as an fMRIPrep derivatives directory, with the original dataset included as the `sourcedata/` sub-dataset.
The following command shows just the directories for the first three subjects. (Will probably not work with earlier versions than `tree 1.8.0`.)

In [ ]:
tree -P "sub-0[123]*.nii.gz" -I "figures|log" --prune --noreport ds003_fmriprep | grep -v -- '->'

### Model

We will estimate the model found in `../models/ds000003/models/model-001_smdl.json`:

In [ ]:
cat ../models/ds000003/models/model-001_smdl.json

## Running FitLins

A BIDS App takes a BIDS dataset, a target directory and an analysis level. Use the BIDS-Raw dataset (`ds003_fmriprep/sourcedata`) as the dataset, and we'll create a new directory `output`. The `dataset` level indicates the highest level of analysis, so `run`, `session` and `subject` will be run, if found in the model.

In addition, we pass `ds003_fmriprep` as with the `--derivatives` (or `-d`) flag, indicating that preprocessed data should be found there. We will also pass the model from `openfmri/ds000003/model.json`

In [ ]:
fitlins ds003_fmriprep/sourcedata output/ dataset \
    --derivatives $PWD/ds003_fmriprep \
    --model $PWD/../models/ds000003/models/model-001_smdl.json \
    --smoothing 5:run \
    -w scratch \
    --n-cpus 2